## Structure Function Coupling

##### Author: A. Zarkali
##### Date of last update: 14/03/2020
##### Aim: Extract cortical-gradients and structure-function coupling scores for each connection

In [1]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import scipy.stats as stats
import statsmodels.api as sm
import seaborn as sns

In [3]:
### Participant Lists
# Load clinical information
clin = pd.read_excel(r"C:\Users\Angelika\Dropbox\PhD\PAPERS\Publications\Structure-Function\NatureComms\SupplementaryData1\Figure2\Data\ClinicalInformation.xlsx")
# Select controls only
controls = clin[clin.PD==0].ParticipantNumber
HighVis = clin[(clin.PD == 1) & (clin.PD_VisPerf == 0)].ParticipantNumber
LowVis = clin[(clin.PD == 1) & (clin.PD_VisPerf == 1)].ParticipantNumber
All = clin.ParticipantNumber

In [ ]:
# Calculate Structure Function per connection
#Load data
rootdir = Path(r"C:\Users\Angelika\Dropbox\PhD\EXPERIMENTS\04_StructureFunction\DATA")
df = pd.DataFrame(data=np.zeros((400,0)))

for j in range(len(All)):
    filenameStruct = rootdir / "STRUCTURE" / All[j] / "shaefer_connectome_norm_400.csv"
    filenameFunc = rootdir / "FUNCTION" / "RawConnectomes400" / (str(All[j]) + "_FuncConnectome_400_norm_bin.csv")
    struct = pd.read_csv(filenameStruct, index_col=0)
    func = pd.read_csv(filenameFunc, index_col=0)
    func = func.replace([np.inf], 0)

    # calculate spearman correlation for each region
    cor = [] # empty list to hold correlations
    pval = [] # empty list to hold p values
    for l in range(400):
        v1 = struct.iloc[l] # vector of structural connectivity for a region
        v2 = func.iloc[l] # vector of structural connectivity for the same region
        # select non-zero elements only
        ind = []
        for l in range(400):
            if (v1[l] != 0) and (v2[l] != 0):
                ind.append(l)
        v1=v1[ind]
        v2=v2[ind]
        # spearman correlation
        rho, p = stats.spearmanr(v1, v2)
        cor.append(rho) # append to correlation list
        pval.append(p) # append to p value list
    # FDR correction
    fdr = sm.stats.multipletests(pval, is_sorted=False, alpha=0.05, method="fdr_bh", returnsorted=False)
    # Combine to a dataframe
    df = pd.DataFrame(cor, columns=["Rho"], index=[k for k in range(1,401)])
    df["Pvalue"] = pval
    df["FDR"] = fdr[1]
    cor=np.array(cor)
    df[str(All[j])] = cor
# Save to file
df.to_csv("StructureFuction_Spearman_Bin.csv")